In [38]:
import cv2
import numpy as np
from PIL import Image
import os
import psutil

# Get current CPU usage as a percentage
cpu_usage = psutil.cpu_percent()

# Get current memory usage
memory_usage = psutil.virtual_memory()

# Convert memory usage object to a dictionary
memory_usage_dict = dict(memory_usage._asdict())

# Print results


In [39]:
data = []
labels = []
dir = "dataset"
detector = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
directorynames = os.listdir(dir)  # directory names inside parent
for directoryname in directorynames:
    if directoryname.startswith("s"):
        x = directoryname.replace("s", "")
    label = int(x)
    directorypath = dir + "/" + directoryname  # directory path

    imagenames = os.listdir(directorypath)
    for imagename in imagenames:
        if not imagename.startswith("."):
            imagepath = directorypath + "/" + imagename  # image path

            img = cv2.imread(imagepath)

            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(src=img, dsize=(100, 100))
            img = np.array(img)
            data.append(img)
            labels.append(label)
data = np.array(data)
labels = np.array(labels)
data = data.reshape((-1, 100, 100, 1))
print(labels)
X_train = data / 255

[1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 3 4 4 4 4 4 4 5 5 5 5]


In [40]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
trainY = lb.fit_transform(labels)

In [41]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [43]:
Model = Sequential()
shape = (100, 100, 1)
Model.add(Conv2D(32, (5, 5), padding="same", input_shape=shape))
Model.add(Activation("relu"))
Model.add(Conv2D(32, (5, 5), padding="same"))
Model.add(Activation("relu"))
Model.add(MaxPooling2D(pool_size=(2, 2)))
Model.add(Conv2D(64, (3, 3), padding="same"))
# Model.add(Conv2D(32, (3, 3), padding="same"))

Model.add(Activation("relu"))
Model.add(MaxPooling2D(pool_size=(2, 2)))
Model.add(Flatten())
Model.add(Dense(128))
Model.add(Activation("relu"))
Model.add(Dense(5))
Model.add(Activation("softmax"))
Model.summary()
Model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print("start training")
Model.fit(X_train, trainY, batch_size=2, epochs=20,validation_split=0.2)
print("Complete training")
Model.save("khuonmat.h5")
print(f"Current CPU usage: {cpu_usage}%")
print(f"Current memory usage: {memory_usage_dict}")

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_84 (Conv2D)          (None, 100, 100, 32)      832       
                                                                 
 activation_144 (Activation)  (None, 100, 100, 32)     0         
                                                                 
 conv2d_85 (Conv2D)          (None, 100, 100, 32)      25632     
                                                                 
 activation_145 (Activation)  (None, 100, 100, 32)     0         
                                                                 
 max_pooling2d_62 (MaxPoolin  (None, 50, 50, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_86 (Conv2D)          (None, 50, 50, 64)        18496     
                                                     